## Прогноз виручки (three-phase linear)

Зошит повторює ключові кроки `3p_linear_model`: базовий Holt-Winters, сезонні та лагові ознаки, фінальна модель XGBoost для кожної товарної групи.

In [1]:
from pathlib import Path

import numpy as np
import pandas as pd

from three_phase_linear import ForecastConfig, run_three_phase_forecast

DATA_PATH = Path('forecast_revenue_dataset.csv')
OUTPUT_PATH = Path('money_three_phase_forecast.csv')
GROUP_COLS = ['category_id']
TARGET_COLUMN = 'revenue'
REGRESSORS = ['is_sale_prohibition', 'cos_month', 'sin_month', 'cos_quarter', 'sin_quarter', 'unique_brand_count']

In [2]:
df = pd.read_csv(DATA_PATH, sep=';')
df['date'] = pd.to_datetime(df['date'], dayfirst=True)
df = df.sort_values(GROUP_COLS + ['date']).reset_index(drop=True)
for col in REGRESSORS:
    df[col] = pd.to_numeric(df[col], errors='coerce')

agg_df = df.groupby(['date', 'category_id', 'category_title'], as_index=False).agg(
    revenue_sum=('revenue', 'sum'),
    revenue_count=('revenue', 'count'),
    is_sale_prohibition=('is_sale_prohibition', 'max'),
    cos_month=('cos_month', 'mean'),
    sin_month=('sin_month', 'mean'),
    cos_quarter=('cos_quarter', 'mean'),
    sin_quarter=('sin_quarter', 'mean'),
    unique_brand_count=('unique_brand_count', 'mean'),
)
agg_df.loc[agg_df['revenue_count'] == 0, 'revenue_sum'] = np.nan
df = agg_df.rename(columns={'revenue_sum': 'revenue'}).drop(columns=['revenue_count'])
df = df.sort_values(GROUP_COLS + ['date']).reset_index(drop=True)

future_counts = df[df[TARGET_COLUMN].isna()].groupby(GROUP_COLS).size()
forecast_horizon = int(future_counts.max()) if not future_counts.empty else 12
if forecast_horizon <= 0:
    forecast_horizon = 12

print(f'Горизонт прогнозу: {forecast_horizon} періодів')

Горизонт прогнозу: 18 періодів


In [3]:
input_cols = ['date', *GROUP_COLS, TARGET_COLUMN, *REGRESSORS]
input_cols = list(dict.fromkeys(input_cols))
config = ForecastConfig(
    time_col='date',
    target_col=TARGET_COLUMN,
    group_cols=GROUP_COLS,
    freq='MS',
    forecast_horizon=forecast_horizon,
    seasonal_periods=12,
    min_history=24,
    lags=(1, 2, 3, 6, 12, 18, 24),
    rolling_windows=(3, 6, 12, 24),
    additional_regressors=REGRESSORS,
    random_search_iterations=10,
    n_splits=4,
    random_state=46,
)

preds, summaries = run_three_phase_forecast(df[input_cols].copy(), config)
preds = preds.rename(columns={
    'prediction': 'revenue_forecast',
    f'{TARGET_COLUMN}_holtwinters': 'revenue_baseline',
})
summary_report = pd.DataFrame({
    'group_key': [s.group_key[0] for s in summaries],
    'train_rows': [s.train_rows for s in summaries],
    'cv_mae': [s.best_score for s in summaries],
    'skipped_reason': [s.skipped_reason for s in summaries],
})
summary_report.head()

/home/jawdypus/Documents/Orders/10 Running Model 2/Veronika/.venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/home/jawdypus/Documents/Orders/10 Running Model 2/Veronika/.venv/lib/python3.12/site-packages/statsmodels/tsa/holtwinters/model.py:903: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/home/jawdypus/Documents/Orders/10 Running Model 2/Veronika/.venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/home/jawdypus/Documents/Orders/10 Running Model 2/Veronika/.venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


,group_key,train_rows,cv_mae,skipped_reason
0,1,86,2.357594e+07,None
1,2,86,2.266108e+07,None
2,3,86,7.842562e+07,None
3,4,86,1.723254e+07,None
4,5,86,1.260030e+07,None


In [4]:
merge_cols = [*GROUP_COLS, 'date']
result_df = df.copy()
result_df['is_forecast_period'] = result_df[TARGET_COLUMN].isna()
result_df = result_df.merge(preds[merge_cols + ['revenue_forecast']], on=merge_cols, how='left')
result_df['revenue'] = result_df['revenue'].astype(float)
result_df['revenue'] = result_df['revenue'].fillna(result_df['revenue_forecast'])

output_columns = ['date', 'category_id', 'category_title', 'revenue']
final_output = result_df.loc[result_df['is_forecast_period'], output_columns]
final_output = final_output.groupby(['date', 'category_id', 'category_title'], as_index=False)['revenue'].sum()
final_output = final_output.sort_values(['category_id', 'date']).reset_index(drop=True)
final_output.to_csv(OUTPUT_PATH, index=False)

final_output.tail()

,date,category_id,category_title,revenue
85,2026-04-01,5,Vodka,115137136.0
86,2026-05-01,5,Vodka,114684784.0
87,2026-06-01,5,Vodka,115319888.0
88,2026-07-01,5,Vodka,106354256.0
89,2026-08-01,5,Vodka,108417216.0


In [5]:
summary_report

,group_key,train_rows,cv_mae,skipped_reason
0,1,86,2.357594e+07,None
1,2,86,2.266108e+07,None
2,3,86,7.842562e+07,None
3,4,86,1.723254e+07,None
4,5,86,1.260030e+07,None
